In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
yrs=[1983,1984,1985]
arr=[]
links=[]
for yr in yrs:
    html=requests.get("https://www.profootballarchives.com/"+str(yr)+"usfl.html")
    soup=BeautifulSoup(html.text)
    stats=soup.find("div",{"id":"stats"})
    table=stats.find_all("table")[0]
    for a in table.find_all("a"):
        links.append(a.get("href"))
        url="https://www.profootballarchives.com/"+str(a.get("href"))
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[4]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(yr)
            arr.append(h)
yrs=[2022]
links=[]
html=requests.get("https://www.profootballarchives.com/2022usflbir.html")
soup=BeautifulSoup(html.text)
table=soup.find_all("table")[3]
for a in table.find_all("a"):
    if '<a></a>' not in str(a):
        links.append(a.get("href"))
for link in links:
    for yr in yrs:
        
        link=str(link)
        if '83' in link:
            link=link.split("83")
            link=link[1]
        if '84' in link:
            link=link.split("84")
            link=link[1]
        if '85' in link:
            link=link.split("85")
            link=link[1]
        link=link
        url="https://www.profootballarchives.com/"+link
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[3]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(2022)
            arr.append(h)
df=pd.concat([pd.DataFrame(arr)])
df.columns=['Day','Date','VenueType','team2','score','result','OT','Location','Venue','Attendance','team1','season']
df=df.astype({'VenueType':'string'})

df=df[df.Date.notna()]
df=df[df.score.notna()]
df['score2']=0
df['score1']=0
for row in df.itertuples():
    score=str(row.score)
    score=score.split("-")
    s1=score[0]
    s2=score[1]
    df.at[row.Index,'score1']=s1
    df.at[row.Index,'score2']=s2
df['Date']=df['Date'].str.replace("/","")
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,"Date":"string"})  
df['team1']=df['team1'].str.replace(" ","")
df['team2']=df['team2'].str.replace(" ","")
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len(f)>0:
        df=df.drop(row.Index)


KeyboardInterrupt: 

In [37]:
import math
import numpy as np
df=pd.read_csv("https://github.com/easella/usfl/raw/main/usflscores.csv")
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,'Date':"string"})
df['team1']=df['team1'].str.replace(" ","")
df['team2']=df['team2'].str.replace(" ","")
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len(f)>0:
        df=df.drop(row.Index)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
homef=65
w=0

df.sort_values(by='Date',inplace=True)
l=0
class Elo:
  global winnerHome
  def __init__(self,k,g=1,homefield = 182.5):
    self.ratingDict   = {}  
    self.k        = k
    self.g        = g
  def addPlayer(self,name,rating = 1500):
    self.ratingDict[name] = rating
  def gameOver(self, winner, loser, winnerHome,neutral,wins,loses):
    homef=50
    wb=0
    lb=0
    if winnerHome and neutral!='N':
      result = self.expectResult(self.ratingDict[winner] + homef, self.ratingDict[loser])
    if winnerHome==False and neutral!='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser] + homef)
    if neutral=='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser])
    wins=float(wins)
    loses=float(loses)
    if wins==loses:
        tie=True
    else:
        tie=False
    if tie:
        mult=(math.log(0+1))*(2.2/1*0.001+2.2)
        shift=(self.k*mult)*(0.5 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
    if tie !=True:
        win=eloLeague.ratingDict[winner]
        lose=eloLeague.ratingDict[loser]
        if neutral==True:
          win=(eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome and neutral!=True:
          win=(homef+eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome!=True and neutral!=True:
          lose=(homef+eloLeague.ratingDict[loser])
          win=(eloLeague.ratingDict[winner])
        global zy
        if(win>lose):
          zy=(win-lose)
        if(win==lose):
          zy=(win-lose)
        if(win<lose):
          zy=(win-lose)
        mult=(math.log((wins-loses)+1))*(2.2/(zy)*0.001+2.2)
        shift=(self.k*mult)*(1 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
  def expectResult(self, p1, p2):
        global exp
        exp = (p2-p1)/400
        n2=10
        j=exp
        o=1/((n2**(j))+1)
        return float(o)
teams=set(df.team1.tolist()+df.team2.tolist())
eloLeague=Elo(k=20)
currSeason=1983
for team in teams:
    eloLeague.addPlayer(team,rating=1500)
for row in df.itertuples():
    if float(row.season)>currSeason:
        season=float(row.season)
        lsdf=df[df.season<season]
        ls=set(lsdf.team1.tolist()+lsdf.team2.tolist())
        csdf=df[df.season==season]
        cs=set(csdf.team1.tolist()+csdf.team2.tolist())
        
                
       
            
       
        
        
        for key in eloLeague.ratingDict.keys():
            
            if(key  in ls and key in cs):
                eloLeague.ratingDict[key]=(eloLeague.ratingDict[key]*(2/3))+(1500*(1/3))
                
        currSeason=float(row.season)
    if row.score1>row.score2 or row.score1==row.score2:
        if row.VenueType=='A':
            hfa=0
            if row.VenueType!='N':
                hfa=65
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]+hfa):
                l+=1
            winnerhome=False
        else:
            hfa=0
            if row.VenueType!='N':
                hfa=65
            if (eloLeague.ratingDict[row.team1]+hfa)>(eloLeague.ratingDict[row.team2]):
                w+=1
            if (eloLeague.ratingDict[row.team1]+hfa)<(eloLeague.ratingDict[row.team2]):
                l+=1
            winnerhome=True
        eloLeague.gameOver(row.team1,row.team2,winnerhome,row.VenueType,row.score1,row.score2)
    if row.score1<row.score2:
        if row.VenueType=='A':
            hfa=0
            if row.VenueType!='N':
                hfa=65
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                l+=1
            winnerhome=True
        else:
            winnerhome=False
        eloLeague.gameOver(row.team2,row.team1,winnerhome,row.VenueType,row.score2,row.score1)
css=df[df.season==2023]
if len(css)==0:
    lsdf=df[df.season<2023]
    ls=set(lsdf.team1.tolist()+lsdf.team2.tolist())
    csdf=df[df.season<2023]
    cs=set(csdf.team1.tolist()+csdf.team2.tolist()) 
    for key in eloLeague.ratingDict.keys():
            
        if(key  in ls and key in cs):
            pre=eloLeague.ratingDict[key]
            eloLeague.ratingDict[key]=(eloLeague.ratingDict[key]*(2/3))+(1500*(1/3))
            af=eloLeague.ratingDict[key]
            print(pre,af,key)
    

1233.9603013343183 1322.6402008895454 WashingtonFederals
1508.2155712085869 1505.4770474723912 HoustonGamblers
1456.4296665690674 1470.9531110460448 NewOrleansBreakers
1618.7572937194977 1579.1715291463317 BostonBreakers
1348.7126811464104 1399.1417874309402 PortlandBreakers
1806.085016437385 1704.0566776249234 BaltimoreStars
1315.6059463867314 1377.0706309244874 SanAntonioGunslingers
1356.1919218382063 1404.1279478921374 ChicagoBlitz
1146.9508837798396 1264.633922519893 PittsburghMaulers
1360.2535337201957 1406.835689146797 MichiganPanthers
1192.3973898816478 1294.9315932544318 LosAngelesExpress
1711.116334705893 1640.744223137262 ArizonaWranglers
1704.9994423804972 1636.6662949203314 NewJerseyGenerals
1193.8392159402008 1295.8928106268004 OklahomaOutlaws
1700.7375049714415 1633.8250033142942 MemphisShowboats
1601.0005639692458 1567.3337093128305 JacksonvilleBulls
1393.7984059900723 1429.1989373267147 TampaBayBandits
1767.792764812695 1678.5285098751299 OaklandInvaders
1794.8346999408

In [33]:
csdf

,Unnamed: 0,Day,Date,VenueType,team2,score,result,OT,Location,Venue,Attendance,team1,season,score2,score1
141,952,Sat,2022-04-16,A,BirminghamStallions,24-28,L,NaN,"Birmingham, AL",Protective Stadium,NaN,NewJerseyGenerals,2022.0,28.0,24.0
147,1058,Sun,2022-04-17,A,MichiganPanthers,17-12,W,NaN,"Birmingham, AL",Protective Stadium,NaN,HoustonGamblers,2022.0,12.0,17.0
146,1096,Sun,2022-04-17,A,NewOrleansBreakers,17-23,L,NaN,"Birmingham, AL",Protective Stadium,NaN,PhiladelphiaStars,2022.0,23.0,17.0
149,1070,Mon,2022-04-18,A,PittsburghMaulers,17-3,W,NaN,"Birmingham, AL",Protective Stadium,NaN,TampaBayBandits,2022.0,3.0,17.0
168,1021,Fri,2022-04-22,A,NewJerseyGenerals,6-10,L,NaN,"Birmingham, AL",Protective Stadium,NaN,MichiganPanthers,2022.0,10.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,1107,Sat,2022-06-25,A,NewJerseyGenerals,19-14,W,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,PhiladelphiaStars,2022.0,14.0,19.0
418,1055,Sat,2022-06-25,A,BirminghamStallions,17-31,L,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,NewOrleansBreakers,2022.0,31.0,17.0
419,1093,Sat,2022-06-25,A,BirminghamStallions,17-31,L,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,NewOrleansBreakers,2022.0,31.0,17.0
472,1017,Sun,2022-07-03,H,BirminghamStallions,33-30,W,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,PhiladelphiaStars,2022.0,30.0,33.0


In [222]:
hfa=[]
n=df[df.VenueType!='N']
for row in n.itertuples():
    if row.VenueType=='A':
        hfa.append(row.score2-row.score1)
    else:
        hfa.append(row.score1-row.score2)
    

In [253]:
print(100/(w+l)*w)

69.68838526912181


In [39]:
l2s={k: v for k, v in sorted(eloLeague.ratingDict.items(), key=lambda item: item[1],reverse=True)}
for key in l2s.keys():
    print(key,eloLeague.ratingDict[key])

BaltimoreStars 1704.0566776249234
PhiladelphiaStars 1696.5564666272608
BirminghamStallions 1683.1538213834062
OaklandInvaders 1678.5285098751299
ArizonaWranglers 1640.744223137262
NewJerseyGenerals 1636.6662949203314
MemphisShowboats 1633.8250033142942
BostonBreakers 1579.1715291463317
JacksonvilleBulls 1567.3337093128305
HoustonGamblers 1505.4770474723912
ArizonaOutlaws 1494.0314838402794
DenverGold 1471.416987680022
NewOrleansBreakers 1470.9531110460448
TampaBayBandits 1429.1989373267147
OrlandoRenegades 1418.6970173511872
MichiganPanthers 1406.835689146797
ChicagoBlitz 1404.1279478921374
PortlandBreakers 1399.1417874309402
SanAntonioGunslingers 1377.0706309244874
WashingtonFederals 1322.6402008895454
OklahomaOutlaws 1295.8928106268004
LosAngelesExpress 1294.9315932544318
PittsburghMaulers 1264.633922519893
